## Purpose 

The purpose of the notebook is to use annotatoins (and possibly description) of each emoji to intelligently cluster emojis
* use webscraping csv file to get dataframe with emoji, annotation...

** Import Packages **

In [67]:
# import packages work bitch please meow
# meow meow
# meow
import nltk, re
import numpy as np
import gensim
from gensim.models import Word2Vec
from nltk.data import find
import pandas as pd
from collections import defaultdict
from nltk.corpus import stopwords
import json
import data_cleaning_hr as tc

** Function to convert the text file into a list of 1) titles 2) descriptions 3) annotations **

In [8]:
df = pd.read_json("data/emoji_webscraped_expanded.json")

In [52]:
df.head()

,annotations,byteCode,byteCode1,byteCode2,descriptions
0,"[face, grin, person]",U+1F600,\U0001F600,,grinning face
1,"[eye, face, grin, person, smile]",U+1F601,\U0001F601,,grinning face with smiling eyes
10,"[bright, cool, eye, eyewear, face, glasses, pe...",U+1F60E,\U0001F60E,,smiling face with sunglasses
100,"[gua pi mao, hat, man, person]",U+1F472,\U0001F472,,man with gua pi mao
1000,"[chinese, ideograph, secret, symbol, word]",U+3299,\U00003299,,circled ideograph secret


*keep old script just in case!!!*

In [9]:
# def convert_scraped_txt(txt):
#     with open("data/emoji_webscraped_expanded.json") as f_in:
# #     with open("emoji_webscraped.txt") as f_in:
#         titles = []
#         descriptions = []
#         annotations = []
#         for line in f_in:
#             line = line.strip()
#             temp = line.split(", ")
#             titles.append(temp[0])
#             descriptions.append(temp[1])
#             annotations.append(temp[2:len(temp)])
#         return titles, descriptions, annotations

# titles, descriptions, annotations = convert_scraped_txt("emoji_webscraped.txt")

In [12]:
# d = {'titles' : (titles),
#      'annotations' : (annotations),
#      'descriptions': (descriptions)}
# df = pd.DataFrame(d)

In [13]:
# len(df.titles.iloc[16])

In [14]:
# p = re.compile('(\+)')
# df.titles = df.titles.apply(lambda x: p.sub('000', x) if len(x) >= 7 else p.sub('',x))

In [15]:
# df[12:17]

In [16]:
# print('\U0000263A')

In [17]:
# for item in df.titles:
#     print(bytes("\\{0}".format(item), 'ascii').decode('unicode-escape'))

**Subset Dataframe to only annotations which either contain face or person**

In [18]:
def subset_annotations(_df):
    list_titles = [list(item) for item in list(_df)]
    index_face_person = [index for index,value in enumerate(list_titles) if 'face' in value] # or 'person' in value]
    # print(len(index_face_person))
    df_face_person = df.iloc[index_face_person]
    # print(df_face_person.shape)
    # df_face_person.head()
    return df_face_person

In [19]:
df_face_person = subset_annotations(df.annotations)

In [20]:
df_face_person.head()

,annotations,byteCode,byteCode1,byteCode2,descriptions
0,"[face, grin, person]",U+1F600,\U0001F600,,grinning face
1,"[eye, face, grin, person, smile]",U+1F601,\U0001F601,,grinning face with smiling eyes
10,"[bright, cool, eye, eyewear, face, glasses, pe...",U+1F60E,\U0001F60E,,smiling face with sunglasses
103,"[aid, cross, face, hat, helmet, person]",U+26D1,\U000026D1,,helmet with white cross
108,"[angel, baby, face, fairy tale, fantasy, person]",U+1F47C,\U0001F47C,,baby angel


** return list of all annotations and the fifty top annotations **

In [21]:
def word_list(annotations, _common=100):
    stop_words = stopwords.words('english')
    total_list = [word for item in list(annotations) for word in item if word not in stop_words]
    top_list = nltk.FreqDist(total_list).most_common(_common)
    return total_list, top_list

In [22]:
total_list, top_list = word_list(df_face_person.annotations, 50)

In [56]:
# total_list[:10], top_list[:10]

** return list of fifty top words without frequency rate **

In [23]:
def top_justwords(top_list):
    top_words = [item[0] for item in top_list]
    return top_words

In [24]:
top_justwords = top_justwords(top_list)

In [25]:
df_features = pd.DataFrame(columns=top_justwords)

In [26]:
top_justwords_array = [(top_justwords) for i in range(df_face_person.shape[0])]
# df_face_person['top_words'] = 1
df_face_person['top_words'] = pd.Series(top_justwords_array, index=df_face_person.index)

/Users/carlo_liquido/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [27]:
df_face_person.shape

(121, 6)

In [28]:
# something = [list(pd.Series(item).isin(df_face_person.annotations)) for item in df_face_person.top_words]

df_face_person['top_binary'] = [list(pd.Series(item).isin(list(df_face_person.annotations)[index])) for index, item in enumerate(df_face_person.top_words)]
df_face_person = df_face_person.reset_index(drop=True)
df_face_person.shape




/Users/carlo_liquido/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


(121, 7)

In [29]:
data_features = list(df_face_person.top_binary)

df_features = pd.DataFrame(data_features, columns=top_justwords)
df_features = df_features.astype(int)
df_features.shape

(121, 50)

In [30]:
df_combined = pd.concat([df_face_person, df_features], axis=1, join_axes=[df_face_person.index])

In [31]:
df_combined.shape

(121, 57)

In [35]:
df_combined.head()

,annotations,byteCode,byteCode1,byteCode2,descriptions,top_words,top_binary,face,person,nature,...,quarter,sun,japanese,speak,angel,alien,joy,fearful,tired,pig
0,"[face, grin, person]",U+1F600,\U0001F600,,grinning face,"[face, person, nature, animal, smile, eye, fai...","[True, True, False, False, False, False, False...",1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,"[eye, face, grin, person, smile]",U+1F601,\U0001F601,,grinning face with smiling eyes,"[face, person, nature, animal, smile, eye, fai...","[True, True, False, False, True, True, False, ...",1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,"[bright, cool, eye, eyewear, face, glasses, pe...",U+1F60E,\U0001F60E,,smiling face with sunglasses,"[face, person, nature, animal, smile, eye, fai...","[True, True, False, False, True, True, False, ...",1,1,0,...,0,1,0,0,0,0,0,0,0,0
3,"[aid, cross, face, hat, helmet, person]",U+26D1,\U000026D1,,helmet with white cross,"[face, person, nature, animal, smile, eye, fai...","[True, True, False, False, False, False, False...",1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,"[angel, baby, face, fairy tale, fantasy, person]",U+1F47C,\U0001F47C,,baby angel,"[face, person, nature, animal, smile, eye, fai...","[True, True, False, False, False, False, True,...",1,1,0,...,0,0,0,0,1,0,0,0,0,0


In [32]:
# df_face_person['top_binary_num'] = None

# for index in range((df_face_person.shape)[0]):
#     temp = []
#     for boolean in df_face_person['top_binary'][index]:
#         if boolean==True:
#             temp.append(1)
#         else:
#             temp.append(0)
#     df_face_person['top_binary_num'][index] = temp

# # for index, _list in enumerate(df_face_person['top_binary']):
# #     temp = []
# #     for boolean in _list:
# #         if boolean==True:
# #             temp.append(1)
# #         else:
# #             temp.append(0)
# #     df_face_person['top_binary_num'][index] = temp


** Notes from John:**
count vectorizer on annoations descriptions
clustering with binary data, possibly asocaition rules
tfidf
feature vector
k_means on either full vector (or on lower dimensional space)

In [45]:
from sklearn.cluster import KMeans

def Clustering(df_combined, n_clusters=5):
    _X = np.array(df_combined.ix[:,7:(df_combined.shape)[1]])
    k_means = KMeans(init='k-means++', n_clusters= n_clusters, n_init=20)
    k_means.fit(_X)
    k_means_labels = k_means.labels_
    k_means_cluster_centers = k_means.cluster_centers_
    k_means_labels_unique = np.unique(k_means_labels)
    ft = (k_means_labels, k_means_cluster_centers, k_means_labels_unique)
    labels = np.array(k_means_labels_unique)
    location = np.array(k_means_cluster_centers)
    labels_location = list(zip(labels, location))
    # person_df['cluster_label'] = pd.DataFrame(k_means_labels)
    print ("labels:\n %s, \n cluster centers:\n %s,\n  unique labels:\n %s" % ft)
    print(labels_location)
    return k_means_labels

k_means_labels = Clustering(df_combined, n_clusters=5)

labels:
 [1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 3 1 1 1 1 4 4 1 4 2 4 4 4 4 1 4 4 4 1 4 4 4
 1 4 4 1 4 4 1 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 0 0 1 0 0 1 4 1 1 1 1 1 1 1 1 1 1 1 3 3 1 3 3 3 1 3 3 3 1 4 2 1 2 2 1
 2 2 2 2 2 2 4 4 1 4], 
 cluster centers:
 [[  1.00000000e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    1.00000000e+00   1.00000000e+00   1.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   8.00000000e-01   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   4.00000000e-01   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    4.00000000e-01   2.

In [46]:
print(df_combined.shape)
print(len(k_means_labels))

(121, 58)
121


In [47]:
df_combined['k_means'] = list(k_means_labels)

In [48]:
unique_labels = list(k_means_labels)
dict_grouping = dict.fromkeys(unique_labels)
dict_grouping

{0: None, 1: None, 2: None, 3: None, 4: None}

In [49]:
subset = df_combined[['k_means','byteCode1']]
subset = subset.values.tolist()
# subset

In [62]:
dict_grouping = defaultdict(list)
for key, date in subset:
    dict_grouping[key].append(date)

In [66]:
type(dict(dict_grouping))

# df_dict_grouping = pd.DataFrame(dict_grouping)

# df_dict_grouping.to_json("k_means_clustering.json")

dict

In [117]:
for key, value in dict_grouping.items():
    print(key)
    for _emoji in value:
        print(bytes("{0}".format(_emoji), 'ascii').decode('unicode-escape'), end="")
    print("\n")
    
#     emojis = list(map(lambda x: bytes("{}{}".format(*x), 'ascii').decode('unicode-escape'), zip(list(emojis_df.byteCode1), list(emojis_df.byteCode2))))
#     for emoji in emojis:
#         print(emoji, end="")


0
🌚🌛🌜🌝🌞

1
😀😁😎⛑😍😘🗣😗😙😚☺👀🙂🤗😂🤔😐😑😶🙄😏😣😥😮🤐😃😯😪😫😴😌🤓😛😜😝☹😄🙁😒😓😔😕😖🙃😷🤒🤕😅🤑😲😞😟😤😢😭😦😧😨😆😩😬😰😱😳😵😡😠😉☠🤖😊🗿😋

2
🐱😺😸😹😻😼😽🙀😿😾

3
👼😇😈👿👹👺💀👻👽👾

4
🐵🐶🐺🦁🐯🐴🦄🐮🐷🐽🐭🐹🐰🐻🐼🐸🐲🐳🌬💩🙈🙉🙊



## See Distribution

In [131]:
clean_tweets_df = tc.loader('./data/tweets_training.json')

In [132]:
tweet_df = pd.DataFrame(clean_tweets_df)
tweet_df.head()

,id,lat,lng,text,timeStamp,user_id
0,383560784879833088,37.369106,-121.998494,"I'm at SEPTEMBER♥ (Sunnyvale, CA) http://t.co/ICHmlfq3FR",Fri Sep 27 11:56:31 +0000 2013,47880944
1,385148713460453376,37.002203,-122.058052,oops. fell asleep in the library. class started at 2.,Tue Oct 01 21:06:23 +0000 2013,259008472
2,380062755639676928,37.438669,-121.879072,@nteimoorian you told me your birthday yourself 2chainz lol,Tue Sep 17 20:16:36 +0000 2013,630468267
3,384797380198928384,37.299621,-121.982024,She throw it back like Thursday,Mon Sep 30 21:50:19 +0000 2013,272108001
4,383691693767143425,39.137889,-121.658646,Not ready for this exam,Fri Sep 27 20:36:43 +0000 2013,790458278


In [133]:
emojis = list(map(lambda x: bytes("{}{}".format(*x), 'ascii').decode('unicode-escape'), zip(list(df_face_person.byteCode1), list(df_face_person.byteCode2))))
for emoji in emojis:
    print(emoji, end="")

# faces_bytecodes = df_face_person.byteCode1
# faces_bytecodes = [item for item in list(faces_bytecodes)]

😀😁😎⛑👼😍😘🗣😗😙😚☺👀🙂🤗😇😂🤔😐😑🐵🐶😶🐺🐱🦁🐯🐴🦄🙄🐮🐷🐽😏🐭🐹🐰😣🐻🐼😥🐸🐲😮🐳🤐😃😯😪😫😴😌🤓😛😜😝☹😄🙁😒😓😔😕😖🙃😷🤒🤕😅🤑😲😞😟😤😢🌚🌛🌜😭🌝🌞😦🌬😧😨😆😩😬😰😱😳😵😡😠😈👿😉👹👺💀☠👻👽👾🤖💩😺😊😸😹🗿😻😼😽🙀😿😾🙈🙉😋🙊

In [134]:
for item in list(faces_bytecodes):
    print(bytes("{0}".format(item), 'ascii').decode('unicode-escape'), end="")

😀😁😎⛑👼😍😘🗣😗😙😚☺👀🙂🤗😇😂🤔😐😑🐵🐶😶🐺🐱🦁🐯🐴🦄🙄🐮🐷🐽😏🐭🐹🐰😣🐻🐼😥🐸🐲😮🐳🤐😃😯😪😫😴😌🤓😛😜😝☹😄🙁😒😓😔😕😖🙃😷🤒🤕😅🤑😲😞😟😤😢🌚🌛🌜😭🌝🌞😦🌬😧😨😆😩😬😰😱😳😵😡😠😈👿😉👹👺💀☠👻👽👾🤖💩😺😊😸😹🗿😻😼😽🙀😿😾🙈🙉😋🙊

In [135]:
# try:
#     # Wide UCS-4 build
#     highpoints = re.compile(u'['
#         u'\U0001F300-\U0001F64F'
#         u'\U0001F680-\U0001F6FF'
#         u'\u2600-\u26FF\u2700-\u27BF]+', 
#         re.UNICODE)
# except re.error:
#     # Narrow UCS-2 build
#     highpoints = re.compile(u'('
#         u'\ud83c[\udf00-\udfff]|'
#         u'\ud83d[\udc00-\ude4f\ude80-\udeff]|'
#         u'[\u2600-\u26FF\u2700-\u27BF])+', 
#         re.UNICODE)

In [136]:
# emoji_list = []
# for index, value in enumerate(tweet_df.text):
#     if highpoints.search(value):
#         emoji_list.append((index, value))
# emoji_index = [x[0] for x in emoji_list]
# emoji_df = tweet_df.ix[emoji_index]

In [137]:
# emoji_df.head()

In [191]:
# List of functions for emoji search
faces = re.compile(u'['
        u'\U0001F600-\U0001F64F]',
        re.UNICODE)

# emojis_faces = list(faces_bytecodes)
# emojis_faces = '|'.join(emojis_faces)
# faces = re.compile(u'['u'(%)]'%emojis_faces, re.UNICODE)

# faces = re.compile(u'[{}].format(faces_bytecodes)',
#         re.UNICODE)

# Function that takes a list of text and return text that contains just faces
def just_face(text):
    return (faces.findall(text))
# Function that take a list of text and return text with just emojis
def just_emojis(text):
    return (highpoints.findall(text))
# Function that a list of text and return the number of emojis in the text.
def count_emojis(text):
    return len(highpoints.findall(text))


In [192]:
print('\U0001F600')
just_face('\U0001F600 hello \U0001F600 bitchin fun')

😀


['😀', '😀']

In [193]:
emoji_df["Emoji Count"] = emoji_df["text"].apply(count_emojis)
emoji_df["Text Length"] = emoji_df["text"].apply(lambda x: len(x))
emoji_df = emoji_df[['id', 'text', 'timeStamp', 'user_id', 'Emoji Count', 'Text Length']]
emoji_df.describe()
emoji_df.head()

,id,text,timeStamp,user_id,Emoji Count,Text Length
0,383560784879833088,"I'm at SEPTEMBER♥ (Sunnyvale, CA) http://t.co/ICHmlfq3FR",Fri Sep 27 11:56:31 +0000 2013,47880944,1,56
5,378697562275840000,You guys!!! Help a friend out and follow one if my good friends new accounts? She follows back!!! :D @kyootniall 💕 RT PLEASE,Sat Sep 14 01:51:49 +0000 2013,1062328976,1,124
7,381268490469904384,@donnariguero not funny! Sad 😞😭 bye money 💸. Now you have to drive me places Donna,Sat Sep 21 04:07:46 +0000 2013,29075313,2,82
17,385855209102012416,Crepevine with @Kkeellyycc_ 😍 http://t.co/8B0gqgmQWZ,Thu Oct 03 19:53:45 +0000 2013,548146276,1,53
21,381165589693071360,Back on the court!!! ✌️✌️,Fri Sep 20 21:18:52 +0000 2013,192611656,2,25


In [194]:
reset_df = emoji_df.reset_index(drop=True)
emoji_array = [reset_df.loc[[index]].text.apply(just_emojis) for index in range(len(reset_df))]
full_list = []
for item in emoji_array:
    for emoji in item:
        for sinlge in emoji:
            full_list.append(sinlge)

In [195]:
full_list[1]

'💕'

In [196]:
# full_dict = defaultdict(int)
# for item in full_list:
#     full_dict[item] += 1

In [197]:
# for item in sorted(full_dict.items(), key=lambda x:x[1], reverse=True)[:50]:
#     print(item[0], (float(item[1])/len(full_list))*100)

In [198]:
face_array = [reset_df.loc[[index]].text.apply(just_face) for index in range(len(reset_df))]

In [199]:
face_array[1]

1    []
Name: text, dtype: object

In [157]:
face_array = [reset_df.loc[[index]].text.apply(just_face) for index in range(len(reset_df))]

face_list = []
for item in face_array:
    for emoji in item:
        for sinlge in emoji:
            face_list.append(sinlge)

print("number of tweets with emojis {0}".format(len(full_list)))
print("number of tweets with faces  {0}".format(len(face_array)))
print("percentage of tweets with emojis with faces {0}%".format(round((float(len(face_array))/len(full_list)*100),1)))

number of tweets with emojis 157728
number of tweets with faces  139576
percentage of tweets with emojis with faces 88.5%


In [158]:
face_dict = defaultdict(int)
for item in face_list:
    face_dict[item] += 1

In [159]:
face_array[0]

0    []
Name: text, dtype: object

In [111]:
for item in sorted(face_dict.items(), key=lambda x:x[1], reverse=True)[:50]:
    print(item[0], (float(item[1])/len(full_list))*100)

# LDA

In [539]:
from gensim import corpora, models, similarities 

In [540]:
df_combined.head()

,annotations,descriptions,titles,top_words,top_binary,face,person,nature,animal,smile,...,quarter,wink,mad,pig,pouting,speak,fear,disappointed,alien,k_means
0,"[face, grin, person]",grinning face,U+1F600,"[face, person, nature, animal, smile, eye, fai...","[True, True, False, False, False, False, False...",1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,"[eye, face, grin, person, smile]",grinning face with smiling eyes,U+1F601,"[face, person, nature, animal, smile, eye, fai...","[True, True, False, False, True, True, False, ...",1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,"[face, joy, person, tear]",face with tears of joy,U+1F602,"[face, person, nature, animal, smile, eye, fai...","[True, True, False, False, False, False, False...",1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,"[face, mouth, open, person, smile]",smiling face with open mouth,U+1F603,"[face, person, nature, animal, smile, eye, fai...","[True, True, False, False, True, False, False,...",1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,"[eye, face, mouth, open, person, smile]",smiling face with open mouth and smiling eyes,U+1F604,"[face, person, nature, animal, smile, eye, fai...","[True, True, False, False, True, True, False, ...",1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [541]:
texts= df_combined.annotations

#create a Gensim dictionary from the texts
dictionary = corpora.Dictionary(texts)

#remove extremes (similar to the min/max df step used when creating the tf-idf matrix)
dictionary.filter_extremes(no_below=1, no_above=0.8)

#convert the dictionary to a bag of words corpus for reference
corpus = [dictionary.doc2bow(text) for text in texts]

In [543]:
dictionary

In [550]:
%time lda = models.LdaModel(corpus, num_topics=10, \
                            id2word=dictionary, \
                            update_every=5, \
                            chunksize=10000, \
                            passes=100)

CPU times: user 12.7 s, sys: 20.3 ms, total: 12.7 s
Wall time: 12.7 s


In [551]:
lda.show_topics()

[(0,
  '0.160*person + 0.054*tear + 0.054*body + 0.054*sad + 0.054*cry + 0.037*monster + 0.037*tongue + 0.037*wink + 0.037*death + 0.037*skull'),
 (1,
  '0.220*person + 0.084*mouth + 0.074*smile + 0.067*open + 0.034*eye + 0.026*frown + 0.026*sweat + 0.026*cold + 0.026*heart + 0.017*mad'),
 (2,
  '0.085*grin + 0.065*person + 0.030*surprised + 0.030*moyai + 0.030*statue + 0.030*travel + 0.030*stunned + 0.030*hushed + 0.030*place + 0.030*horse'),
 (3,
  '0.142*person + 0.127*eye + 0.091*kiss + 0.070*smile + 0.020*horrible + 0.020*taste + 0.020*injury + 0.020*bandage + 0.020*hurt + 0.020*blush'),
 (4,
  '0.226*nature + 0.183*animal + 0.074*cat + 0.045*weather + 0.037*place + 0.037*space + 0.030*moon + 0.030*pet + 0.015*quarter + 0.015*pig'),
 (5,
  '0.167*person + 0.098*fairy tale + 0.088*fantasy + 0.059*monster + 0.049*creature + 0.030*smile + 0.021*nature + 0.020*alien + 0.020*extraterrestrial + 0.020*ufo'),
 (6,
  '0.130*person + 0.034*tired + 0.034*joy + 0.034*tear + 0.021*nature + 0.0

In [554]:
topics_matrix = lda.show_topics(formatted=False, num_words=20)
topics_matrix
# topics_matrix = np.array(topics_matrix)

# topic_words = topics_matrix[:,:,1]
# for i in topic_words:
#     print([str(word) for word in i])
#     print()

[(0,
  [('person', 0.15961170571365257),
   ('tear', 0.054049528870208134),
   ('body', 0.054048960919355127),
   ('sad', 0.054048889347755584),
   ('cry', 0.054048889347755501),
   ('monster', 0.036613954044276337),
   ('tongue', 0.036613914152601776),
   ('wink', 0.036613820149575593),
   ('death', 0.03661380121855598),
   ('skull', 0.036613801218555647),
   ('eye', 0.036612959518771401),
   ('sob', 0.019178667725240899),
   ('joke', 0.019178667725222757),
   ('rolling', 0.019178667690230852),
   ('eyes', 0.019178667690229384),
   ('fairy tale', 0.019178116009641769),
   ('crossbones', 0.0017580003381538812),
   ('cat', 0.0017436440999485186),
   ('animal', 0.0017435659004636809),
   ('nature', 0.0017435561814640567)]),
 (1,
  [('person', 0.21984458527397097),
   ('mouth', 0.08379984779038116),
   ('smile', 0.07404616692994731),
   ('open', 0.06720580127213728),
   ('eye', 0.034035309510224009),
   ('frown', 0.02572075675343655),
   ('sweat', 0.025720709808085301),
   ('cold', 0.0257

# Word2Vec

In [603]:
from gensim.models.doc2vec import TaggedDocument, LabeledSentence, Doc2Vec

In [604]:
def label_sentence(_df):
    sentences = []
    for uid, line in enumerate(_df):
        sentences.append(LabeledSentence(words=df_combined.annotations[uid], \
                              tags=[df_combined.descriptions[uid]]))
    return sentences

label_sentence(df_combined)

[LabeledSentence(words=['face', 'grin', 'person'], tags=['grinning face']),
 LabeledSentence(words=['eye', 'face', 'grin', 'person', 'smile'], tags=['grinning face with smiling eyes']),
 LabeledSentence(words=['face', 'joy', 'person', 'tear'], tags=['face with tears of joy']),
 LabeledSentence(words=['face', 'mouth', 'open', 'person', 'smile'], tags=['smiling face with open mouth']),
 LabeledSentence(words=['eye', 'face', 'mouth', 'open', 'person', 'smile'], tags=['smiling face with open mouth and smiling eyes']),
 LabeledSentence(words=['cold', 'face', 'open', 'person', 'smile', 'sweat'], tags=['smiling face with open mouth and cold sweat']),
 LabeledSentence(words=['face', 'laugh', 'mouth', 'open', 'person', 'satisfied', 'smile'], tags=['smiling face with open mouth and tightly-closed eyes']),
 LabeledSentence(words=['face', 'person', 'wink'], tags=['winking face']),
 LabeledSentence(words=['blush', 'eye', 'face', 'person', 'smile'], tags=['smiling face with smiling eyes']),
 Labeled

In [605]:
sentences = list(LabeledSentence(df_combined.annotations[uid], (df_combined.descriptions[uid])) for uid, value in enumerate(df_combined))
model = Doc2Vec(size=200, min_count=1, workers=16)
model.build_vocab(sentences)


# model = Doc2Vec(min_count=1, window=10, size=100, sample=1e-4, negative=5, workers=8)

# model.build_vocab(sentences.to_array)

In [607]:
for epoch in range(10):
    model.train(sentences)
    model.alpha -= 0.002  # decrease the learning rate
    model.min_alpha = model.alpha

In [609]:
# sims = model.docvecs.most_similar(['grinning face'])
# model.most_similar('grinning face')
model.most_similar("face with thermometer")

KeyError: "word 'face with thermometer' not in vocabulary"